In [28]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install seaborn
import seaborn as sns
from datetime import *
import datetime as dt
from matplotlib.ticker import MaxNLocator
#!pip install regex
import regex
#!pip install emoji
import emoji
from seaborn import *
#!pip install heatmap
#from heatmap import heatmap
#!pip install wordcloud
#from wordcloud import WordCloud , STOPWORDS , ImageColorGenerator
from nltk import *
#from plotly import express as px

### Python code to extract Date from chat file 

In [55]:
def startsWithDateAndTime(s):
    print('Start with Date and Time function')
    print(s)
    # 11/08/2021, 7:21 pm -
    # pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
    pattern = '[0-9]{2}/[0-9]{2}/[0-9]{4}'
    #p = re.compile(pattern)
    #r = p.match(s)
    #print('r -> ', r)
    result = re.match(pattern, s)
    #print(result)
    if result:
        return True
    return False

### Regex pattern to extract username of Author.

In [56]:

def FindAuthor(s):
    # ~~~~ Produce error ~~~~~ #
    patterns = [
        '([w]+):',                        # First Name
        '([w]+[s]+[w]+):',              # First Name + Last Name
        '([w]+[s]+[w]+[s]+[w]+):',    # First Name + Middle Name + Last Name
        '([+]d{2} d{5} d{5}):',         # Mobile Number (India no.)
        '([+]d{2} d{3} d{3} d{4}):',   # Mobile Number (US no.)
        '([w]+)[u263a-U0001f999]+:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    
    result = re.match(pattern, s)
    if result:
        return True
    return False

### Extracting Date, Time, Author and message from the chat file.

In [57]:

def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    #if FindAuthor(message): 
    splitMessage = message.split(': ') 
    author = splitMessage[0] 
    message = ' '.join(splitMessage[1:])
    #else:
    #    author = None
    return date, time, author, message


In [58]:
### Finally creating a dataframe and storing all data inside that dataframe.
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = 'GroupChat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline()
    fp.readline()
    fp.readline()
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        #print(line)
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            print(100)
            if len(messageBuffer) > 0:
                print('!!!!!!!!!!!!!!!!!')
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message) 
        else:
            #print(200)
            messageBuffer.append(line)
    
print(len(parsedData))

df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

Start with Date and Time function
11/08/2021, 7:21 pm - HR: Why you created this Group😄
100
Start with Date and Time function
11/08/2021, 7:21 pm - Saad MS_34: 😂
100
!!!!!!!!!!!!!!!!!
Start with Date and Time function
11/08/2021, 7:22 pm - HR: Rockstar ko b add kr ly
100
!!!!!!!!!!!!!!!!!
Start with Date and Time function
11/08/2021, 7:22 pm - HR: 😏
100
!!!!!!!!!!!!!!!!!
Start with Date and Time function
11/08/2021, 7:22 pm - Saad MS_34: 😂😂😂
100
!!!!!!!!!!!!!!!!!
Start with Date and Time function
11/08/2021, 7:23 pm - HR: Otherwise me jaa rha classmate
100
!!!!!!!!!!!!!!!!!
Start with Date and Time function
11/08/2021, 7:23 pm - HR: I want Rockstar here
100
!!!!!!!!!!!!!!!!!
Start with Date and Time function
11/08/2021, 7:23 pm - HR: 🙌🏼
100
!!!!!!!!!!!!!!!!!
Start with Date and Time function
11/08/2021, 7:23 pm - Saad MS_34: 😂😂😂
100
!!!!!!!!!!!!!!!!!
Start with Date and Time function
11/08/2021, 7:24 pm - Saad MS_34: Me krda gal
100
!!!!!!!!!!!!!!!!!
Start with Date and Time function
1

In [59]:
df

,Date,Time,Author,Message
0,2021-11-08,7:21 pm,HR,Why you created this Group😄
1,2021-11-08,7:21 pm,Saad MS_34,😂
2,2021-11-08,7:22 pm,HR,Rockstar ko b add kr ly
3,2021-11-08,7:22 pm,HR,😏
4,2021-11-08,7:22 pm,Saad MS_34,😂😂😂
...,...,...,...,...
1139,2021-08-14,2:04 pm,HR,Now Iqra on form 😄
1140,2021-08-14,2:12 pm,HR,Relx rahy Mam
1141,2021-08-14,2:12 pm,HR,🙌🏼
1142,2021-08-14,2:17 pm,Iqra MS_33,Her insan apny decision leny ka haq rkhta h so...


1240


11/08/2021
7:21 pm
HR
Why you created this Group😄
<re.Match object; span=(0, 10), match='11/08/2021'>
yes
